In [35]:
import tabula.io as ti
import pandas as pd
import re
import os
import sys
import time

pd.options.mode.chained_assignment = None  # default='warn'

In [77]:
##This frontpage functions works on none suspenderade reports 
def frontPage(myFrontpage):
    df = myFrontpage
    colID=df.filter(regex=("Provnummer.*")).columns.to_list()
    startRow=df.loc[(df[colID[0]] == 'Analys')|(df[colID[0]] == 'Analys ')].index.to_numpy()
    dfCh=df.iloc[startRow[0]+1:df.shape[0],:]
    dfSt=df.iloc[0:startRow[0]-1:,:]
    dfSt = dfSt.astype("str")
    x1=dfSt.apply(lambda x: x.str.contains("^[0-9]{4}-[0-9]{2}-[0-9]{2}", regex = True), axis = 1)    
    dateDf=dfSt.loc[x1.apply(lambda x:any(x), axis = 1),x1.apply(lambda x:any(x))] #Use two apply functions to identify the true position in row and column index
    dateVal = dateDf.iloc[0,0]
    mtchPrv=re.search("(?<=Provnummer: ).*",df.columns.to_list()[0])
    dfCh.rename(columns={dfCh.columns[0]: "Analys"}, inplace=True)
    dfCh=dfCh.drop(dfCh.filter(regex='^Unnamed:.+$'), axis =1)
    dfCh.iloc[:,1]=dfCh.loc[:,"Ankomsttemp °C Kem"].str.replace("[0-9]{2}%$","", regex = True)
    dfCh.iloc[:,1]=dfCh.loc[:,"Ankomsttemp °C Kem"].str.replace("< ", "<")
    dfCh.iloc[:,1]=dfCh.loc[:,"Ankomsttemp °C Kem"].str.replace("\s[0-9]\.[0-9]$","",regex = True)
    dfCh[['Result', "Method"]]=dfCh.iloc[:,1].str.split(" ", 1, expand = True)
    topFrame=dfCh.loc[:,['Analys','Result','Method']]
    topFrame.loc[topFrame["Analys"] == "pH","Method"] = "." #Adds a "." to prevent pH-method to be interpreted as null
    topFrame['Analys'] = topFrame['Analys']+"_"+topFrame['Method']
    topFrame = topFrame.transpose()
    topFrame.columns = topFrame.iloc[0,:]
    topFrame = topFrame.iloc[[1],:]
    topFrame.reset_index(inplace=True, drop=True)
    metaFrame = pd.DataFrame({"Provnum":mtchPrv.group(),"Date":dateVal}, index=[0])
    topFrame = pd.concat([metaFrame, topFrame], axis = 1) # Need to have same/compatibel index to work? Ignore index = True?
    return(topFrame)

#Create function for secondary pages 
def SecondaryPage(mySecpage):
    scDf = mySecpage
    scDf.loc[-1] = scDf.columns
    scDf.index = scDf.index + 1
    scDf.sort_index(inplace=True)
    scDf.rename(columns={scDf.columns[0] :"Analys", scDf.columns[1]:"Result", scDf.columns[2]:"Method"}, inplace=True)
    scDf = scDf.loc[:,["Analys", "Result", "Method"]]
    scDf['Analys']=scDf['Analys']+"_"+scDf['Method']
    scDf = scDf.dropna()
    scDf = scDf.transpose()
    scDf.reset_index(inplace=True, drop = True)
    scDf.columns = scDf.iloc[0,:]
    scDf = scDf.iloc[[1],:]
    scDf.reset_index(inplace=True, drop = True)
    return scDf


def tableAdjustment(aFrame):
    if(any(aFrame.columns.str.contains(r"^Provnummer:$"))):
        print("We got in!")
        prvID = aFrame.columns.str.contains(r"^Provnummer:$")
        numID = aFrame.columns.str.contains(r"^[0-9]{1,3}-[0-9]{4}-[0-9].+$")
        numStr = aFrame.loc[:,numID].squeeze().name
        prvStr = aFrame.loc[:,prvID].squeeze().name
        aFrame[numStr] = aFrame[numStr].fillna('')
        aFrame[prvStr] = aFrame[prvStr].fillna('')
        aFrame[prvStr] = aFrame[prvStr] +" "+aFrame[numStr]
        aFrame.rename(columns={prvStr:(aFrame.columns[0]+" "+aFrame.columns[1])}, inplace = True)
        aFrame.drop(labels=numStr, axis = 1, inplace =True)
        return aFrame
    else:
        return aFrame

In [121]:
myPath = "C:/Calluna/Data/SandBox/Convert_PDF_Table/Analyssvar/Fria vattenmassan 2022/Juli/Påverkansområde CePå" # Set pdf-folder to run script against
os.chdir(myPath) 
myFolder=myPath.split("/")[-1]

In [112]:
pd.set_option("display.max_columns", 500)

In [122]:
fileFilter = [x for x in os.listdir(myPath) if x.endswith(".pdf")]

In [123]:
myPdfs=pd.DataFrame()

for aPdf in fileFilter:
    print(f"Analysing:\n{aPdf}")
    secPages = pd.DataFrame()
    pdfNm=re.search(".+?(?=_[0-9]{3})",aPdf).group()
    logicGate = True
    pdfPath = myPath+"/"+aPdf
    dfs = ti.read_pdf(pdfPath, pages = "all")
    for aTbl in range(0, len(dfs)):
        if(logicGate):
            #print("First!")
            dfs[aTbl] = tableAdjustment(dfs[aTbl])
            myFront = frontPage(dfs[aTbl])
            watChmCol = myFront.columns[2:myFront.shape[1]]
            myFront['Name'] = re.search(".+?(?=_[0-9]{3})",aPdf).group()
            myFront = myFront.loc[:,~myFront.columns.duplicated()].copy()
            myFront = myFront.reindex(["Provnum", "Date","Name"] + list(watChmCol),axis =1)
            logicGate = False
        else:
            #print("Secondary!")
            mySec = SecondaryPage(dfs[aTbl])
            secPages=pd.concat([secPages, mySec], axis =1)
    myTables = pd.concat([myFront, secPages], axis =1)
    myTables = myTables.loc[:,~myTables.columns.duplicated()].copy()
    myTables.columns = myTables.columns.fillna("MissingName")
    myEnd = myTables.reindex(sorted(myTables.columns[3:myTables.shape[1]],key=str.casefold), axis = 1)
    myTables = pd.concat([myTables.iloc[:,[0,1,2]], myEnd], axis =1)
    myPdfs = pd.concat([myPdfs, myTables], axis = 0)
myPdfs.to_csv(f"Vattenkemi_FranPdf_{myFolder}.csv", index =False)

Analysing:
CePa_A. botten_177-2022-07210237_01.pdf
We got in!
Analysing:
CePa_A. mitten_177-2022-07210235_01.pdf


C:\Users\MartinAndersson\AppData\Local\Temp\ipykernel_12628\3926853610.py:18: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dfCh[['Result', "Method"]]=dfCh.iloc[:,1].str.split(" ", 1, expand = True)


We got in!
Analysing:
CePa_A. yta_177-2022-07210338_01.pdf


C:\Users\MartinAndersson\AppData\Local\Temp\ipykernel_12628\3926853610.py:18: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dfCh[['Result', "Method"]]=dfCh.iloc[:,1].str.split(" ", 1, expand = True)


We got in!
Analysing:
CePa_C. botten_177-2022-07210225_01.pdf


...

C:\Users\MartinAndersson\AppData\Local\Temp\ipykernel_12628\3926853610.py:18: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dfCh[['Result', "Method"]]=dfCh.iloc[:,1].str.split(" ", 1, expand = True)


We got in!
Analysing:
CePa_Y. yta_177-2022-07210334_01.pdf


C:\Users\MartinAndersson\AppData\Local\Temp\ipykernel_12628\3926853610.py:18: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dfCh[['Result', "Method"]]=dfCh.iloc[:,1].str.split(" ", 1, expand = True)


We got in!


C:\Users\MartinAndersson\AppData\Local\Temp\ipykernel_12628\3926853610.py:18: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  dfCh[['Result', "Method"]]=dfCh.iloc[:,1].str.split(" ", 1, expand = True)


In [124]:
myPdfs

,Provnum,Date,Name,"1,2,3,4,6,7,8-HeptaCDD_pg/l","1,2,3,4,6,7,8-HeptaCDF_pg/l","1,2,3,4,7,8,9-HeptaCDF_pg/l","1,2,3,4,7,8-HexaCDD_pg/l","1,2,3,4,7,8-HexaCDF_pg/l","1,2,3,6,7,8-HexaCDD_pg/l","1,2,3,6,7,8-HexaCDF_pg/l","1,2,3,7,8,9-HexaCDD_pg/l","1,2,3,7,8,9-HexaCDF_pg/l","1,2,3,7,8-PentaCDD_pg/l","1,2,3,7,8-PentaCDF_pg/l","2,3,4,6,7,8-HexaCDF_pg/l","2,3,4,7,8-PentaCDF_pg/l","2,3,7,8-TetraCDD_pg/l","2,3,7,8-TetraCDF_pg/l",6:2 FTS (Fluortelomer sulfonat)_ng/l,Acenaften _μg/l,Acenaftylen _μg/l,Alifater >C10-C12 _mg/l,Alifater >C12-C16 _mg/l,Alifater >C12-C35 _mg/l,Alifater >C16-C35 _mg/l,Alifater >C5-C12 _mg/l,Alifater >C5-C8 _mg/l,Alifater >C8-C10 _mg/l,Alkalinitet_mg HCO3/l,Antracen_μg/l,Aromater >C10-C16 _mg/l,Aromater >C16-C35 _mg/l,Aromater >C8-C10 _mg/l,Arsenik As (end surgjort)_mg/l,Arsenik As (filtrerat)_mg/l,Barium Ba (end surgjort)_mg/l,Barium Ba (filtrerat)_mg/l,Bens(a)antracen _μg/l,Bensen _mg/l,Benso(a)pyren _μg/l,"Benso(b,k)fluoranten _μg/l","Benso(g,h,i)perylen_μg/l",Bly Pb (end surgjort)_mg/l,Bly Pb (filtrerat)_mg/l,"Dibens(a,h)antracen _μg/l",DOC_mg/l,Etylbensen _mg/l,Fenantren_μg/l,Fluoranten_μg/l,Fluoren_μg/l,Fosfor P_μg/l,I-TEQ (NATO/CCMS) exkl LOQ_pg/l,I-TEQ (NATO/CCMS) inkl LOQ_pg/l,"Indeno(1,2,3-cd)pyren _μg/l",Kadmium Cd (end surgjort)_mg/l,Kadmium Cd (filtrerat)_mg/l,Kobolt Co (filtrerat)_mg/l,"Kobolt, Co (end surgjort)_mg/l",Konduktivitet_mS/m,Koppar Cu (end surgjort)_mg/l,Koppar Cu (filtrerat)_mg/l,Krom Cr (end surgjort)_mg/l,Krom Cr (filtrerat)_mg/l,Krysen _μg/l,Kvicksilver Hg (filtrerat)_mg/l,Kvicksilver Hg (uppslutet)_mg/l,M/P/O-Xylen _mg/l,MissingName,Naftalen _μg/l,Nickel Ni (end surgjort)_mg/l,Nickel Ni (filtrerat)_mg/l,OktaCDD_pg/l,OktaCDF_pg/l,PCB 105_pg/l,PCB 114_pg/l,PCB 118_pg/l,PCB 123_pg/l,PCB 126_pg/l,PCB 156_pg/l,PCB 157_pg/l,PCB 167_pg/l,PCB 169_pg/l,PCB 189_pg/l,PCB 77_pg/l,PCB 81_pg/l,PFBA (Perfluorbutansyra)_ng/l,PFBS (Perfluorbutansulfonsyra)_ng/l,PFDA (Perfluordekansyra)_ng/l,PFHpA (Perfluorheptansyra)_ng/l,PFHxA (Perfluorhexansyra)_ng/l,PFHxS (Perfluorhexansulfonsyra)_ng/l,PFNA (Perfluornonansyra)_ng/l,PFOA (Perfluoroktansyra)_ng/l,PFOS (Perfluoroktansulfonsyra)_ng/l,PFPeA (Perfluorpentansyra)_ng/l,Pyren_μg/l,Summa cancerogena PAH _μg/l,Summa PAH med hög molekylvikt_μg/l,Summa PAH med låg molekylvikt_μg/l,Summa PAH med medelhög molekylvikt_μg/l,Summa PFAS SLV 11_ng/l,Summa PFAS4_ng/l,Summa övriga PAH_μg/l,Suspenderade ämnen_mg/l,Temperatur vid pH-mätning_°C,Toluen _mg/l,Total-kväve_μg/l,Turbiditet_FNU,Vanadin V (filtrerat)_mg/l,"Vanadin, V (end surgjort)_mg/l",WHO(2005)-PCB TEQ exkl. LOQ_pg/l,WHO(2005)-PCB TEQ inkl. LOQ_pg/l,WHO(2005)-PCDD/F TEQ exkl. LOQ_pg/l,WHO(2005)-PCDD/F TEQ inkl. LOQ_pg/l,Zink Zn (end surgjort)_mg/l,Zink Zn (filtrerat)_mg/l,Alkalinitet _mg HCO3/l,Fosfor P _μg/l,Konduktivitet _mS/m,Suspenderade ämnen _mg/l,Temperatur vid pH-mätning _°C,Total-kväve _μg/l,Turbiditet _FNU,Unnamed: 0_Unnamed: 2,Vattentemperatur vid provtagning_°C,Alkalinitet_mg HCO3/l,Fosfor P_μg/l,Konduktivitet_mS/m,pH_.,Suspenderade ämnen_mg/l,Total-kväve_μg/l,Turbiditet_FNU
0,177-2022-07210237,2022-07-21 04:48,CePa_A. botten,< 1.59,< 1.48,< 1.48,< 1.86,< 1.55,< 1.86,< 1.55,< 1.86,< 1.55,< 0.932,< 1.67,< 1.55,< 1.67,< 0.699,< 1.24,<0.30,<0.010,<0.010,<0.020,<0.020,<0.050,<0.050,<0.030,<0.020,<0.020,96,< 0.010,<0.010,<0.0050,<0.010,0.00083,0.00083,0.021,0.021,<0.010,<0.00050,<0.010,<0.020,< 0.010,< 0.000010,< 0.000010,<0.010,4.2,<0.0010,< 0.010,< 0.010,< 0.010,13,ND,3.45,<0.010,0.0000090,0.0000070,0.000011,0.000015,1200,0.00042,0.00036,0.000089,< 0.000050,<0.010,< 0.00010,< 0.00010,<0.0010,Utgår,<0.020,0.00053,0.00054,< 11.3,< 3.11,< 75.7,< 9.13,< 272,< 7.77,< 4.47,< 42.7,< 7.96,< 21.4,< 23.3,< 7.77,< 35,< 4.66,0.93,<0.30,<0.30,0.35,<0.30,<0.30,<0.30,0.41,0.31,0.32,< 0.010,<0.20,< 0.040,< 0.040,< 0.040,2.3,0.72,< 0.30,6.0,22.2,<0.0010,230,0.28,0.000068,0.00019,ND,1.16,ND,3.54,0.00026,0.00032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,177-2022-07210235,2022-07-21 04:46,CePa_

In [76]:
#dyFront = myFront 
#dyFront.columns.dropna(axis = 1)

#dyFront=dyFront.reindex(["Provnum", "Date","Name"] + list(watChmCol),axis =1)
#myTables
#dfs = ti.read_pdf(pdfPath, pages = "all")

Error from tabula-java:
Error: File does not exist




CalledProcessError: Command '['java', '-Dfile.encoding=UTF8', '-jar', 'C:\\Users\\MartinAndersson\\anaconda3\\envs\\pandasTutor\\lib\\site-packages\\tabula\\tabula-1.0.5-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'JSON', 'C:/Calluna/Data/SandBox/Convert_PDF_Table/Analyssvar/2022-06-08_09/CePå_A mitten_177-2022-06100824_01.pdf']' returned non-zero exit status 1.

In [75]:
print(pdfPath)
aPdf

C:/Calluna/Data/SandBox/Convert_PDF_Table/Analyssvar/2022-06-08_09/CePå_A mitten_177-2022-06100824_01.pdf


'CePå_A mitten_177-2022-06100824_01.pdf'

In [106]:
myFolder

'Juli\\Kontrollområde CeKo'

In [125]:
%pwd

'C:\\Calluna\\Data\\SandBox\\Convert_PDF_Table\\Analyssvar\\Fria vattenmassan 2022\\Juli\\Påverkansområde CePå'